In [1]:
# !conda install -c conda-forge earthengine-api -y

In [18]:
from IPython.core.display import display, HTML
display(HTML('<h1><center>Farm Monitor</center></h1>'))
display(HTML('<p> <b>INSTRUCTIONS:</b> Click on the pentagon icon and make a polygon outlining your field. Then let Farm Monitor do its magic!</p>'))

In [19]:
import subprocess
from datetime import datetime
from geemap import DrawControl

try:
        import geemap
except ImportError:
        print('geemap/pyshp package not installed. Installing ...')
        subprocess.check_call(["python", '-m', 'pip', 'install', 'geemap'])

import geemap as emap
from ipygee import *

# Authenticates and initializes Earth Engine
import ee

try:
        ee.Initialize()
except Exception as e:
        ee.Authenticate()
        ee.Initialize()



In [20]:
from ipygee import *
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go


from ipywidgets import HTML
from ipyleaflet import Popup

import ipywidgets as widgets
import pandas as pd
from datetime import datetime
from datetime import timedelta
from ipyleaflet import WidgetControl

In [21]:
# !pip install earthengine-api
# !pip install plotly
# !pip install ipyleaflet
# !pip install ipywidgets
# !pip install geemap
# !jupyter nbextension list
# !pip install ipygee

In [22]:
import numpy as np

def get_popup(geometry_object, start_date, end_date):

    region = geometry_object
    imageData = ee.ImageCollection("LANDSAT/LC08/C01/T1_TOA").filterDate(start_date, end_date)
    imageData = imageData.map(lambda image : image.clip(geometry_object))
    imageData = imageData.filterBounds(region)

    imageData = imageData.map(cloudlessNDVI)
    imageData = imageData.map(cloudlessMoistureIndex)
    imageData = imageData.map(get_pH)
    ndmi_ph = None
    ndvi_val=None
    ndmi_val = None
   
    # NDVI first
    ndvi_df = make_values(start_date, end_date, imageData, region, "NDVI").dataframe
    ndvi_val = np.mean(ndvi_df['NDVI'])
   
    # NDMI second
    ndmi_df = make_values(start_date, end_date, imageData, region, "NDMI").dataframe
    ndmi_val = np.mean(ndmi_df['NDMI'])
    
    # pH finally
    ndmi_ph = make_values(start_date, end_date, imageData, region, "pH").dataframe
    ph_val = np.mean(ndmi_ph['pH'])
   
    centroid = geometry_object.centroid().getInfo()
    center = centroid['coordinates']
    temp = center[0]
    center[0] = center[1]
    center[1] = temp
   
    if ndvi_val and ndmi_val and ph_val:
       # print("yay")
        message1 = HTML()
        message1.value = "Vegetation Index: {:.4f},<br> Moisture Index:{:.4f},<br> pH value:{:.4f}".format(ndvi_val, ndmi_val,ph_val)

        popup = Popup(
            location=center,
            child=message1,
            close_button=True,
            auto_close=True,
            close_on_escape_key=False
        )

        Map.add_layer(popup)
    else:
        print("No PopUp Generated")

In [23]:
def add_shape_files(Map):
    file_names = ["QSC_Extracted_Data_Foxdale","QSC_Extracted_Data_GlenIsla","QSC_Extracted_Data_HamiltonPlains","QSC_Extracted_Data_KelseyCreek","QSC_Extracted_Data_Myrtlevale","QSC_Extracted_Data_Proserpine"]

    for files in file_names:
        qld_properties_shp = files + "/Property_boundaries___DCDB_Lite.shp"
        qld_properties = geemap.shp_to_ee(qld_properties_shp)
        Map.addLayer(qld_properties, {}, files.split('_')[3])

In [24]:
def make_values(startDate, endDate, imageCollection, region, band_type):
    imageCollection = imageCollection.select(band_type)
    chart_ts = chart.Image.series(**{
        'imageCollection': imageCollection, 
        'region': region,
        'scale': 50,
        'band': band_type,
        'label_bands':[band_type],
        'reducer':ee.Reducer.mean()
    })

    return chart_ts

In [25]:
import ee 
from ipygee import *
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go


def get_time_series_chart(features, startDate, endDate, band_to_add = "NDVI"):
    
#     print("MAKING Chart for : ", band_to_add)
    
    plotData = pd.DataFrame()
    fig = go.Figure()
    
    fig.update_layout(
    title= band_to_add + " values over the years",
    xaxis_title="Time",
    yaxis_title=band_to_add,
    )
    
    feature_count = 0
    for feature in features:
        feature_count += 1

        region = ee.Geometry(feature['geometry'])
    
        # filter the imageCollection to images of the given range
        imageData = ee.ImageCollection("LANDSAT/LC08/C01/T1_TOA").filterDate(startDate, endDate)

        # clip the images to the given region
        imageData = imageData.filterBounds(region)

        if band_to_add == "NDVI":
            # add the NDVI band and select it
            imageData = imageData.map(cloudlessNDVI)
            imageData = imageData.select('NDVI');
            
        elif band_to_add == "NDMI":
            # add the NDMI band and select it
            imageData = imageData.map(cloudlessMoistureIndex)
            imageData = imageData.select('NDMI');
            
        elif band_to_add == "pH":
            # add the NDMI band and select it
            imageData = imageData.map(get_pH)
            imageData = imageData.select('pH');
            
        else:
            print("Invalid parameter for band_to_add:", band_to_add)
            return fig

        # get the data from the selected imageCollection
        chartData = make_values(startDate, endDate, imageData, region, band_to_add)
        chartData = chartData.dataframe
    
        # converting the labels to the required date format
        y = [pd.to_datetime(tm) for tm in chartData.index]
        labels = [x.strftime('%b %e, %y') for x in y]
        chartData["Time"] = labels
        if band_to_add == "NDVI":
            chartData = chartData[["Time", "NDVI"]]
            chartData.rename(columns={"NDVI": "NDVI-" + str(feature_count)}, inplace = True)
        elif band_to_add == "NDMI":
            chartData = chartData[["Time", "NDMI"]]
            chartData.rename(columns={"NDMI": "NDMI-" + str(feature_count)}, inplace = True)
        elif band_to_add == "pH":
            chartData = chartData[["Time", "pH"]]
            chartData.rename(columns={"pH": "pH-" + str(feature_count)}, inplace = True)

        if plotData.empty:
            plotData = chartData
        else:
            plotData = pd.merge(plotData, chartData, on="Time")

    index = 1
    for i in range(len(plotData.columns) - 1):
        # create the plot
        if band_to_add == "NDVI":
            fig.add_trace(go.Scatter(x=plotData["Time"], y = plotData.iloc[:,index], mode="lines", name = "NDVI-" + str(index)))
        elif band_to_add == "NDMI":
            fig.add_trace(go.Scatter(x=plotData["Time"], y = plotData.iloc[:,index], mode="lines", name = "NDMI-" + str(index)))
        elif band_to_add == "pH":
            fig.add_trace(go.Scatter(x=plotData["Time"], y = plotData.iloc[:,index], mode="lines", name = "pH-" + str(index)))
        index += 1
    return fig



In [26]:
import subprocess
from datetime import datetime
from datetime import timedelta
from geemap import DrawControl
from ipywidgets import HTML
from ipyleaflet import Popup

try:
        import geemap
except ImportError:
        print('geemap/pyshp package not installed. Installing ...')
        subprocess.check_call(["python", '-m', 'pip', 'install', 'geemap'])

import geemap as emap
from ipygee import *

# Authenticates and initializes Earth Engine
import ee

try:
        ee.Initialize()
except Exception as e:
#         ee.Authenticate()
        ee.Initialize()

# Creates an interactive map
Map = emap.Map(center=[-20,148], zoom=10)

draw_control = DrawControl()

# USGS Landsat 8 Collection 1 Tier 1 TOA Reflectance from EARTH ENGINE data catalog
l8 = ee.ImageCollection("LANDSAT/LC08/C01/T1_TOA")

'''NDVI AND NDMI layer generation functions'''


# Calculate the NDVI for the given image while performing cloud masking. 
def cloudlessNDVI(image):
    # Get a cloud score in [0, 100].       
    cloud = ee.Algorithms.Landsat.simpleCloudScore(image).select('cloud');

    # Create a mask of cloudy pixels from an arbitrary threshold.
    mask = cloud.lte(20);

    # Compute NDVI.
    ndvi = image.normalizedDifference(['B5', 'B4']).rename('NDVI');

    # Return the masked image with an NDVI band.
    return image.addBands(ndvi).updateMask(mask);


def cloudlessMoistureIndex(image):
  # Get a cloud score in [0, 100].
    cloud = ee.Algorithms.Landsat.simpleCloudScore(image).select('cloud');

  # Create a mask of cloudy pixels from an arbitrary threshold.
    mask = cloud.lte(20);

  # Compute NDVI.
    moist = image.normalizedDifference(['B5', 'B6']).rename('NDMI');

  # Return the masked image with an NDVI band.
    return image.addBands(moist).updateMask(mask);

def get_pH(image):
    pH_band = image.expression(
        '7.84 + (0.48 * B2) - (0.12 * B6)', {
          'B6': image.select('B6'),
          'B2': image.select('B2')
    }).rename('pH')
    return image.addBands(pH_band)

def generateIndexLayer(geometry_object, count, l8):
#     ndvi_pal = ['#d73027', '#f46d43', '#fdae61', '#fee08b', '#d9ef8b','#a6d96a']
    ndvi_pal = ['FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718', '74A901', '66A000', '529400', '3E8601', '207401', '056201', '004C00', '023B01', '012E01', '011D01', '011301']
    pH_pal = ['fe0002', 'd9fd07', '37a737', '004dff', '44008b' ]
    
    cloudless_masked_NDVI = l8.map(cloudlessNDVI)
    final_NDVI = cloudless_masked_NDVI.map(lambda image : image.clip(geometry_object)) 
#     final_NDVI = cloudless_masked_NDVI.filterBounds(geometry_object) 
    
    cloudless_masked_NDMI = l8.map(cloudlessMoistureIndex)
    final_NDMI = cloudless_masked_NDMI.map(lambda image : image.clip(geometry_object)) 
#     final_NDMI = cloudless_masked_NDMI.filterBounds(geometry_object) 
    
    ph_collection = l8.map(get_pH)
    ph_collection_final = ph_collection.map(lambda image : image.clip(geometry_object)) 
#     ph_collection_final = ph_collection.filterBounds(geometry_object) 
    
    Map.addLayer(final_NDVI.select('NDVI'), {min:-0.5, max:0.9, 'palette': ndvi_pal}, "NDVI - " + str(count))
    Map.addLayer(final_NDMI.select('NDMI'), {min:-0.5, max:0.9, 'palette': ndvi_pal}, "NDMI - " + str(count))
#     Map.addLayer(ph_collection_final.select('pH'), {min:5, max:9, 'palette': pH_pal}, "pH - " + str(count), shown = False, opacity = 0.5)
    
#     centroid = geometry_object.centroid().getInfo()
#     center = centroid['coordinates']
    
#     temp = center[0]
#     center[0] = center[1]
#     center[1] = temp
    
#     message1 = HTML()
#     message1.value = "Vegetation Index: , Moisture Index: ,pH value:"

#     popup = Popup(
#         location=center,
#         child=message1,
#         close_button=False,
#         auto_close=False,
#         close_on_escape_key=False
#     )

#     Map.add_layer(popup)
    

def handle_draw(geo_json, count, l8, start_date, end_date):
    newPolygon = ee.Geometry(geo_json["geometry"])
    generateIndexLayer(newPolygon, count, l8)
    get_popup(newPolygon, start_date, end_date)
    
    
features = []
def add_Feature(self,action, geo_json):
    features.append(geo_json)
    draw_features()
    
        
def draw_features(l8 = ee.ImageCollection("LANDSAT/LC08/C01/T1_TOA"), start_date = datetime(2014, 1, 1), end_date = datetime.now()):
    count = 0
    for feature in features:
        count += 1
        handle_draw(feature, count, l8, start_date, end_date)
    create_index_charts()
    
    
def create_index_charts(startDate = datetime(2014, 1, 1), endDate = datetime.now()):
    if features:
        get_time_series_chart(features, startDate, endDate, "NDVI").show()
        get_time_series_chart(features, startDate, endDate, "NDMI").show()
        get_time_series_chart(features, startDate, endDate, "pH").show()
            
draw_control.polygon = {
    "shapeOptions": {
        "fillColor": "#6be5c3",
        "color": "#6be5c3",
        "fillOpacity": 1.0
    },
    "drawError": {
        "color": "#dd253b",
        "message": "Oops!"
    },
    "allowIntersection": False
}
draw_control.rectangle = {
    "shapeOptions": {
        "fillColor": "#fca45d",
        "color": "#fca45d",
        "fillOpacity": 1.0
    }
}

Map.add_basemap('SATELLITE')
Map.add_control(draw_control)
draw_control.on_draw(add_Feature)


import ipywidgets as widgets
import pandas as pd
from datetime import datetime
from ipyleaflet import WidgetControl

start_date = datetime(2014, 1, 1)
end_date = datetime.now()

dates = pd.date_range(start_date, end_date, freq='SMS')

options = [(date.strftime(' %d %b %y '), date) for date in dates]
index = (0, len(options)-1)

date_range_slider = widgets.SelectionSlider(
    options=options,
    description='Date:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True
)

def print_date_range(date_range):
    l8 = ee.ImageCollection("LANDSAT/LC08/C01/T1_TOA").filterDate(date_range, date_range + timedelta(days=20))
    draw_features(l8, date_range, date_range + timedelta(days=20))
    
widgets.interact(
    print_date_range,
    date_range=date_range_slider
);

widget_control2 = WidgetControl(widget=date_range_slider, position='bottomright')
Map.add_control(widget_control2)
add_shape_files(Map)

Map

interactive(children=(SelectionSlider(continuous_update=False, description='Date:', options=((' 01 Jan 14 ', T…

Map(center=[-20, 148], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(value=…

In [8]:
create_index_charts()